In [1]:
import sys, os
from pathlib import Path
WORKDIR = Path(os.getcwd())
os.chdir(WORKDIR)
os.environ['WORKDIR'] = str(WORKDIR)

In [15]:
!sudo apt install -yy htop ninja-build
#!pip install pytorch-lightning
!conda install pytorch-lightning=1 opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
htop is already the newest version (2.2.0-1+b1).
ninja-build is already the newest version (1.8.2-1).
The following package was automatically installed and is no longer required:
  python3-crcmod
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.



In [5]:
!git clone https://github.com/amikhalev/TinyHITNet.git TinyHITNet
HITNET = WORKDIR / 'TinyHITNet'
sys.path.append(str(HITNET))

Cloning into 'TinyHITNet'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 75 (delta 22), reused 72 (delta 19), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [6]:
!TORCH_CUDA_ARCH_LIST="3.7+PTX" pip install ./TinyHITNet/ext_op/

Processing ./TinyHITNet/ext_op
  Preparing metadata (setup.py) ... - done
  Created wheel for HitnetModule: filename=HitnetModule-0.0.0-cp37-cp37m-linux_x86_64.whl size=2399331 sha256=e25c2debfce7d7d7a43a3fef95a4216ee95a0d2ea707eb794fa3cce01f9c34e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0f0nyju/wheels/7c/53/75/d76e82895fa5974e0be9f13d627c1832ea50cf17245149ddbc
Successfully built HitnetModule


In [10]:
!nvidia-smi

Sun Mar 13 00:54:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    53W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!DIR=kitty2015; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip; \
unzip -qn data_scene_flow.zip
KITTY2015 = WORKDIR / "kitty2015"

--2022-03-13 00:54:40--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.141
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1681488619 (1.6G) [application/zip]
Saving to: ‘data_scene_flow.zip’

data_scene_flow.zip 100%[===================>]   1.57G  21.5MB/s    in 77s     

2022-03-13 00:55:58 (20.9 MB/s) - ‘data_scene_flow.zip’ saved [1681488619/1681488619]



In [12]:
!DIR=kitty2012; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip; \
unzip -qn data_stereo_flow.zip
KITTY2012 = WORKDIR /"kitty2012"

--2022-03-13 00:56:07--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.168.99
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.168.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008641404 (1.9G) [application/zip]
Saving to: ‘data_stereo_flow.zip’

data_stereo_flow.zi 100%[===================>]   1.87G  21.4MB/s    in 90s     

2022-03-13 00:57:38 (21.2 MB/s) - ‘data_stereo_flow.zip’ saved [2008641404/2008641404]



In [19]:
from preprocess import plane_fitting_kitti2012, plane_fitting_kitti2015
import multiprocessing as mp
import tqdm

def preprocess_main(root, list_path, module, n_gpu=1):
    root = Path(root)

    with open(list_path, "rt") as fp:
        file_list = [Path(line.strip()) for line in fp]

    lock_list = [mp.Lock() for _ in range(n_gpu)]
    with mp.Pool(n_gpu, module.process_init, [lock_list, root]) as pool:
        list(tqdm.tqdm(pool.imap_unordered(module.process, file_list), total=len(file_list)))

preprocess_main(KITTY2012 / 'training', HITNET / 'lists' / 'kitti2012_train.list', plane_fitting_kitti2012)
preprocess_main(KITTY2015 / 'training', HITNET / 'lists' / 'kitti2015_train.list', plane_fitting_kitti2015)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.41it/s]


In [22]:
!cd {HITNET}; python3 train.py \
--log_dir logs \
--exp_name hitnet_kitti \
--model HITNet_KITTI \
--check_val_every_n_epoch 5 \
--max_steps 500000 \
--max_disp 256 \
--max_disp_val 192 \
--optmizer Adam \
--lr 4e-4 \
--lr_decay 400000 0.25 408000 0.1 410000 0.025 \
--lr_decay_type Lambda \
--batch_size 4 \
--batch_size_val 4 \
--num_workers 2 \
--num_workers_val 2 \
--data_augmentation 1 \
--data_type_train KITTI2012 KITTI2015 \
--data_root_train {KITTY2012}/training {KITTY2015}/training \
--data_list_train lists/kitti2012_train170.list lists/kitti2015_train180.list \
--data_size_train 1152 320 \
--data_type_val KITTI2015 \
--data_root_val {KITTY2015}/training \
--data_list_val lists/kitti2015_val20.list \
--data_size_val 1242 375 \
--init_loss_k 3 \
--gpus -1 \
--accelerator gpu


Global seed set to 2021
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | HITNet_KITTI     | 668 K 
1 | train_metric | MetricCollection | 0     
2 | val_metric   | MetricCollection | 0     
--------------------------------------------------
668 K     Trainable params
0         Non-trainable params
668 K     Total params
2.673     Total estimated model params size (MB)
Validation sanity check: 0it [00:00, ?it/s]/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader,